<a href="https://colab.research.google.com/github/leeCodingStudio/Python_NLP/blob/master/16_DistilBERT%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_MRC_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. MRC(Machine Reading Comprehension)
* 주어진 질문에 정확한 답변을 추출하는 자연어처리 작업
* 컴퓨터가 사람처럼 텍스트를 이해하고 추론하여 정보를 얻고 질문에 맞는 답변을 찾는 과정

In [1]:
!pip install -U transformers datasets scipy scikit-learn evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Su

In [34]:
import datasets
import random
import pandas as pd
from datasets import load_dataset, ClassLabel
from IPython.display import display, HTML
from transformers import AutoTokenizer, pipeline

In [5]:
raw_datasets = load_dataset('klue', 'mrc')

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
raw_datasets['train'][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [20]:
print('context: ', raw_datasets['train'][0]['context'])
print('question: ', raw_datasets['train'][0]['question'])
print('answers: ', raw_datasets['train'][0]['answers'])

context:  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.
question:  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
answers:  {'answer_start': [478, 478], 'text': ['한 달가량', '한 달']}


In [21]:
print('context: ', raw_datasets['validation'][0]['context'])
print('question: ', raw_datasets['validation'][0]['question'])
print('answers: ', raw_datasets['validation'][0]['answers'])

context:  BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.
question:  말라카이트에서 나온 색깔을 사용한 에디션은?
answers:  {'answer_start': [666, 

In [17]:
def show_random_elements(dataset, num_examples=10):
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

In [18]:
show_random_elements(raw_datasets['train'])

,title,context,news_category,source,guid,is_impossible,question_type,question,answers
0,가인지경영,"인재경영은 ‘사람을 성장시켜 그와 함께 일한다’를 테마로 HRM과 HRD분야를 경영하는 원론이다. 돈보다 사람의 성장에 포커스를 맞추어야 한다고 주장하며, 재능과 강점이 발휘되는 조직이 되는 것을 목표로 한다.\n\n가인지경영은 인재 경영의 시작이 인재를 발굴하고 적합한 사람을 확보하는 것이라고 명시한다. 또한, 강점에 초점을 맞추어 강점이 발휘되는 도전적인 일을 부여하면 인재가 성장한다는 기준을 가지고 일을 부여하도록 한다. 개인 성장의 기준은 다른 사람과의 비교가 아닌 자신의 가치에 따른 성장임을 강조하며, 즐겁게 일하는 환경 조성 또한 강조한다. 이론적인 강의보다 현장 경험 통한 성장이 더욱 효과적이라는 원칙을 가지고 현장에서의 학습을 권장한다.\n\n인재경영의 장점 중 하나는 각각 직원의 부가가치가 높아지면서 직원 자체가 부가가치를 창출하는 중요한 자원이 되는 것이다. 또한, 개인의 성장을 위해 배우려는 문화와 성장을 통한 중간 관리자로의 성장, 그리고 외부 영입의 최소화로 인해 체계적인 가치와 지식이 전수되는 문화가 형성된다.",None,wikipedia,klue-mrc-v1_train_04242,True,3,가인지경영은 경험을 통해 학습하는 것 보다 무엇을 더 선호하는가?,"{'answer_start': [312], 'text': ['이론적인 강의']}"
1,천연두,"천연두의 무시무시함으로 인해 구세계 곳곳에는 천연두와 관련된 신(神)들이 숭배되었다. 중국에는 두진낭랑(痘疹娘娘)이라는 천연두의 여신이 있다. 중국인들 중 하층 계급인들은 여신의 심기를 거스르지 않고 그녀의 자비를 구하기 위해 적극적으로 행동했다. 천연두 물집을 “아름다운 꽃”이라고 부르는 완곡어법이 그러한 숭배행위의 사례이다. 섣달 그믐날(새해 전야)에는 어린이에게 두진낭랑이 붙지 않도록 하기 위해 못생긴 가면을 씌우고 재우는 풍습이 있었다 천연두 감염자가 발생하면 환자의 집은 금줄이 쳐짐으로써 일종의 성역이 된다. 환자가 회복되면 금줄을 철거하여 종이로 만든 의자나 배에 싣고 불태운다. 환자가 회복되지 못하면 성역이 파괴되고 두진낭랑을 집에서 쫓아내기 위한 저주가 행해진다\n\n인도에서 천연두에 관한 기록은 기원후 400년경의 의학서까지 거슬러 올라간다. 이 책에서 설명하고 있는 한 질병은 유난히 천연두와 닮았다 인도 역시 중국과 마찬가지로 천연두에 관련된 여신을 만들었다. 시탈라는 경외의 대상이자 동시에 공포의 대상이었다. 이 여신은 악하면서 동시에 선하여, 분노했을 때는 병을 일으키는 한편 병자의 열을 내려 주기도 한다고 믿어졌다. 시탈라 여신을 묘사한 그림들을 보면 오른손에 병을 몰고 다니는 빗자루를 들고 왼손에는 환자들을 진정시켜 주는 냉수 단지를 들고 있다 시탈라를 모시는 사원들이 만들어졌고, 건강한 자나 병든 자나 모두 천연두로부터 보호받기 위해 사원을 찾아 여신을 숭배했다. 일부 인도 여성들은 시탈라를 쫓기 위한 방책으로 지붕 위에 음식을 식히는 접시와 물동이를 올려두기도 했다. \n\n천연두의 신이 존재하지 않는 문화권에서는 천연두를 일으키는 악마가 존재하여 병의 원인으로 지목되었다. 이러한 믿음은 일본(포창신), 유럽, 아프리카를 비롯한 세계 다른 곳들에서 두드러진다. 거의 모든 문화권에서 이러한 천연두 악마는 붉은색을 두려워한다. 하여 환자와 환자의 방을 붉은색으로 꾸미는 요법이 만들어졌고, 이 관습은 12세기에 유럽으로 전래되어 프랑스의 샤를 5세, 잉글랜드의 엘리자베스 1세도 사용했다 이러한 믿음은 1930년대까지 지속되었다.",None,wikipedia,klue-mrc-v1_train_02819,False,2,악한 동시에 선하다고 믿어진 여신은?,"{'answer_start': [496], 'text': ['시탈라']}"
2,유희왕 GX,";사메지마 교장(사민준)\n:성우 : 이와사키 마사미\n:한국판 성우 : 고성일\n:듀얼 아카데미의 교장이며 마루후지 료의 스승. 나중에 헬 카이저가 된 료에게 교훈을 주려고 헬 카이저와 겨루지만 료에게 당하고 부상당하여 크로노스 교감이 교장 대행으로 학교를 운영했으며 완쾌한 후 완전히 교장으로 복귀한다. 사용하는 덱은 사이버 덱이다.\n\n;크로노스 데 메디치\n:성우 : 시미즈 히로시\n:한국판 성우 : 유호한\n:듀얼 아카데미의 교사 중 한 사람으로 오벨리스크 블루 소속이다. 사메지마 교장이 듀얼 아카데미를 중심으로 한 제넥스 개최를 추진하는 등 교장 일을 할 수 없게 되자 임시로 교장 일을 맡게 된다. 사용하는 덱은 앤틱 기어 덱으로, 에이스 카드는 [앤틱 기어 골렘]이지만 1화의 입학 시험에서 쥬다이에게 패배한 이후 쥬다이를 싫어하며 오시리스 레드를 해체하려는 인물. 가끔씩 쥬다이 편을 들며 오시리스 레드를 지키려고 하는 등의 행동으로 나폴레옹 교감과 싸우기도 하는 인물이며 쥬다이가 존경하는 인물 중 한 명이다.\n:듀얼 아카데미의 중역(重役)으로 등장 횟수가 많고 사용하는 덱이 앤틱 기어 덱이라 유희왕 5d's에서는 크로노스의 앤틱 기어 골렘이 듀얼 아카데미의 상징이 되며 유희왕 ARC-V에서는 융합 차원의 아카데미아의 에이스 전용 덱으로도 등장한 바 있다.\n\n;나폴레옹 교감\n:성우 : 다츠타 나오키\n:한국판 성우 : 최석필\n:후기에 듀얼 아카데미에 교감으로 들어온 인물. 오시리스 레드의 학생들을 모두 하등하다고 생각하여 하루빨리 갈아 엎어버리기를 바라지만 쥬다이 때문에 뜻대로 되지 않는 듯. 크로노스 교장 대행과 달리 통통한 체형이며 키가 작다. 메인 덱은 토이 솔져 덱이다. 후에는 쥬다이의 도움으로 아들인 마르탄이 되돌아오자 같이 귀가한다.\n\n;다이토쿠지 선생(아라미스)\n:성우 : 야마구치 캇페이\n:한국판 성우 : 안용욱\n:오시리스 레드 소속 교사로 연금술 담당 교수. 파라오라고 하는 이름의 고양이와 함께 생활하고 있으며 카게마루 회장의 지원을 받아 삼환마 부활 작업에 동조한 적이 있었으나 쥬다이와의 듀얼에서 패해 육체가 소멸되고 영혼만 남았다. 나중에 쥬다이와 함께 생활하기로 결심하고 머무는 인물 .\n\n;아유카와 에미(에이미)\n:듀얼 아카데미 양호 선생이다. 쥬다이 일행들을 비롯한 여러 학생들을 간호한다. 이차원에서는 듀얼 좀비로 변하여 쥬다이와 듀얼을 한다. 이후 듀얼 좀비에서 회복하여 다시 듀얼 아카데미의 양호 선생으로서 여러 학생들을 간호하게 된다.\n\n;카바야마\n:라 옐로우 소속 교사이다. 교사들 중에서 존재감이 떨어진다. 카레를 매우 좋아해서 '카레가면'이라는 별명도 있다. 오시리스 레드가 폐쇄 위기에 처했을 때 켄잔과 쇼, 미사와가 그것을 막으려고 레드 기숙사에서 지내게 되자 돌아오게 하려고 카레로 켄잔을 유혹해 듀얼을 하나 진다.\n\n;프로페서 코브라\n:성우 : 다카즈카 마사야\n:한국판 성우 : 손정성\n:듀얼 아카데미 웨스트 분교의 교수로 듀얼 아카데미에 들어오자마자 스파르타식 듀얼을 강조하며 아카데미를 '데스 듀얼'의 무대로 만들어 버리는 인물. 하지만 오히려 데스 듀얼로 부활한 [유벨]에 의해 남은 생명을 다하게 된다 . 메인 덱은 베놈 덱이다.\n\n;사토 코우지(밀로)\n:성우 : 미야모토 미츠루\n:한국판 성우 : ?\n:듀얼 아카데미의 교사로 전직 프로 듀얼리스트이다. 원래는 프로가 될 정도의 재능을 지녔으나 타이틀 전에서는 체력 부족으로 은퇴하게 된다. 이후 듀얼 아카데미의 교사로 들어오나 쥬다이가 수업 분위기를 망쳐 놓은 것에 대하여 쥬다이한테 원한을 품게 되고 코브라 편에 선다. 쥬다이와의 듀얼에서 쥬다이를 길동무로 삼으려 하나 되려 본인이 패하고 쥬다이한테 자신이 뭘 위해서 듀얼을 하는지 알게 될 거라는 말을 남기고 다리 밑으로 추락사한다.",None,wikipedia,klue-mrc-v1_train_04367,True,3,쥬다이가 프로에서 은퇴하게 만들었던 인물은?,"{'answer_start': [1604, 1604, 1611], 'text': ['사토 코우지(밀로)', '사토 코우지', '밀로']}"
3,start-up 모바일 세차 중개

In [24]:
tokenizer = AutoTokenizer.from_pretrained('bespin-global/klue-bert-base-mrc')

In [25]:
context = raw_datasets['train'][0]['context']
question = raw_datasets['train'][0]['question']

inputs = tokenizer(question, context)
tokenizer.decode(inputs['input_ids'])

'[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20 ~ 21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24 ~ 25일이었으며 장마기간은 32일, 강수일수는 17. 2일이었다. 기상청은 올해 장마기간의 평균 강수량이 350 ~ 400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다. [SEP]'

In [26]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True
)

In [27]:
for ids in inputs['input_ids']:
    print(tokenizer.decode(ids))

[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20 ~ 21일 남부지

In [28]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation='only_second',
    stride=10,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)

In [29]:
for ids in inputs['input_ids']:
    print(tokenizer.decode(ids))

[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20 ~ 21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24 ~ 25일이었으며 장마기간 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]24 ~ 25일이었으며 장마기간은 32일, 강수일수는 17. 2일이었다. 기상청은 올해 장마기간의 평균 강수량이 350 ~ 400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러

In [30]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [31]:
inputs['input_ids'][0][:10]

[2, 1174, 18956, 26797, 2145, 22152, 2128, 2292, 2097, 26797]

In [32]:
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0, 0, 0]

In [33]:
print(inputs['offset_mapping'])

[[(0, 0), (0, 1), (1, 4), (5, 7), (7, 8), (9, 11), (11, 12), (12, 13), (13, 14), (15, 17), (17, 18), (19, 21), (22, 24), (24, 25), (26, 29), (29, 30), (31, 33), (33, 34), (34, 35), (0, 0), (0, 1), (1, 3), (4, 6), (6, 7), (8, 10), (10, 11), (12, 15), (15, 17), (18, 20), (20, 21), (21, 22), (22, 23), (24, 26), (27, 28), (29, 31), (31, 33), (33, 34), (35, 37), (37, 38), (38, 39), (40, 42), (42, 43), (44, 46), (47, 48), (48, 49), (50, 52), (53, 54), (54, 55), (56, 58), (58, 59), (60, 62), (62, 63), (64, 66), (66, 68), (68, 69), (69, 71), (71, 72), (73, 76), (76, 77), (78, 80), (80, 81), (82, 85), (86, 88), (89, 90), (90, 92), (92, 93), (94, 95), (95, 96), (97, 99), (99, 100), (100, 101), (101, 102), (103, 105), (105, 107), (108, 110), (111, 114), (115, 117), (118, 119), (120, 122), (122, 124), (124, 125), (126, 128), (128, 130), (130, 131), (131, 132), (133, 136), (136, 138), (139, 141), (141, 142), (143, 146), (146, 147), (147, 148), (149, 151), (151, 152), (152, 153), (154, 155), (155, 1

In [37]:
question_answerer = pipeline('question-answering', model='domyoung/squad-test')
question_answerer(question=question, context=context)

{'score': 0.5458457469940186, 'start': 478, 'end': 483, 'answer': '한 달가량'}

In [38]:
context = '''
날카로운 포핸드와 자타공인 완벽한 백핸드를 가졌다. 예전에는 수준급 방어력을 앞세운 카운터펀처였으나 현재는 공수 밸런스를 갖춘 완벽한 올라운드 플레이어로 거듭났다.[30] 독보적인 리턴을 보유하고 있어 압도적이지 않은 서브로도 최고의 기량을 과시하는 중이다.[31] 플레이 스타일은 전형적인 어그레시브 베이스라이너로서 네트로 나오는 일은 드물며, 그래서 네트플레이에서 에러가 가끔 있는 편이다. 대신 수비력이 아주 좋아서 어지간한 볼은 전부 다 쳐낸다. 2014년 이후로는 더욱 더 강력해져서 전성기의 나달 수준, 혹은 그 이상의 방어력을 보여주고 있는 괴물 같은 선수. 전성기 조코비치를 한마디로 요약하자면 코트를 가리지 않는 이상적인 베이스라이너(baseliner)라 할 수 있다. 민첩함과 유연함을 모두 갖춘만큼 엄청난 코트 커버리지를 자랑하며 통곡의 벽이 무엇인지 보여준다.[32] 나달이 클레이코트 수비의 정점이라면 하드코트 수비에선 조코비치가 그 위치에 있다. 랠리를 끝낼만한 상대의 훌륭한 샷을 기어코 쫓아가 받아내는 모습을 심심찮게 볼 수 있다. 왕년의 최고의 베이스라이너인 안드레 애거시의 완성형이라는 표현에 고개가 끄덕여진다. 수비력이 뛰어난 선수지만 공격력이 약하다고 생각하면 큰 오산이다. 심리전이 엄청나게 좋아서 상대방의 빈 곳으로 공을 보내는 것에 능하며 기본적으로 뛰어난 포핸드를 가지고 있다. 그러나 조코비치 공격의 진면목은 완벽한 백핸드[33]에 있다. 상대방이 본인의 백핸드 쪽으로 리턴을 하면 백핸드를 상대의 빈 코스 또는 거의 한 손 백핸드에 가까운 각도로 보내는 장면이 나올 정도로 완벽한 백핸드를 가지고 있다. 역대급 코트 커버리지에 더해 이런 사기적인 스트로크를 가진 조코비치를 랠리 싸움에서 이기는 건 여간해선 힘들다. 서비스 게임 당 에이스 개수 0.443개를 보면 알 수 있듯이, 조코비치의 서브는 위력만 놓고 보면 특별하다고 볼 수 없다. 실제로 남자 선수로서 평범하다면 평범할 수 있는 180km 후반에서 190km 중반 수준의 서브 속도를 가지고 있다. 하지만 에이스가 터지지 않아도 정확한 코스 선택으로 이후 이어지는 랠리에서 주도권을 가지고 가며, 뛰어난 세컨드 서브[34] 또한 가지고 있다.
'''
question = '서비스 게임 당 에이스 개수는?'
question_answerer(question=question, context=context)

{'score': 0.7910722494125366, 'start': 892, 'end': 899, 'answer': '0.443개를'}